# Linear regression을 활용한 부동산 예측

## 1.Import Required Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom()


## 2. Data Loader

In [2]:
num_data=1000
num_epoch=200

import pandas as pd # 엑셀 데이터 처리를 위한 pandas패키지 로드

df = pd.read_csv("price.csv", parse_dates =["date"], index_col ="date")  #엑셀 파일을 읽어 df변수에 저장
df = df.dropna()
#df=(df-df.mean())/df.std()
#df=(df-df.min())/(df.max()-df.min())+0.0001

df.head()


,tradeprice_sido_n1,region_cd,tradeprice_sido,year,month,building_type,construction_realized_amount,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,,,,,,,
2010-01-01,93.4,45000,93.3,2010,1,7,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,0.0,0.0
2010-01-01,83.9,30000,83.1,2010,1,0,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,92.7,30000,92.3,2010,1,7,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,80.8,30000,79.7,2010,1,1,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0
2010-01-01,81.8,30000,81.8,2010,1,3,6500576.0,2.88,3.93,1156.5,1682.16,2.2,4.29,548723.3,338666.3,3042.0,323.0


In [3]:
df.resample(rule='A').mean() #연도별로 통계를 내보자

,tradeprice_sido_n1,region_cd,tradeprice_sido,year,month,building_type,construction_realized_amount,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,,,,,,,
2010-12-31,85.678906,36750.000000,85.469271,2010.0,6.500000,2.75,7.689668e+06,2.671667,3.181667,1159.400000,1761.925833,1.475000,3.718333,568365.566667,349610.691667,1236.640625,541.677083
2011-12-31,90.873037,36727.748691,90.302749,2011.0,6.502618,2.75,7.633607e+06,3.437016,3.686492,1107.846597,1983.284188,0.725654,3.620681,615922.894764,377893.788482,683.026178,423.408377
2012-12-31,94.025387,36742.268041,94.015464,2012.0,6.551546,2.75,7.411050e+06,3.293041,3.424485,1124.052577,1931.431649,0.526804,3.130155,645835.318557,396972.789175,454.520619,186.876289
2013-12-31,94.198652,36705.882353,94.098775,2013.0,6.500000,2.75,8.174064e+06,2.722500,2.703333,1095.433333,1961.248333,0.850000,2.793333,666946.983333,406966.191667,547.215686,239.333333
2014-12-31,95.477340,36684.729064,95.375739,2014.0,6.507389,2.75,8.207670e+06,2.490887,2.420099,1052.333990,1982.338030,0.700493,2.591281,710863.112315,436247.054187,248.014778,108.068966
2015-12-31,97.420098,36705.882353,97.220221,2015.0,6.500000,2.75,8.551165e+06,1.770000,1.719167,1133.083333,2010.792500,0.800000,1.799167,772351.316667,476190.525000,180.446078,62.382353
2016-12-31,98.678342,36653.465347,98.643317,2016.0,6.514851,2.75,1.001116e+07,1.493317,1.466188,1163.131188,1986.238416,0.626238,1.441683,859289.918317,518222.086634,341.965347,50.356436
2017-12-31,99.529774,36608.040201,99.435427,2017.0,6.512563,2.75,1.152759e+07,1.444372,1.514221,1122.084422,2312.495829,0.704523,1.805427,939928.797487,561827.164322,541.894472,93.417085
2018-12-31,100.211224,36571.428571,100.162755,2018.0,2.000000,2.75,1.073421e+07,1.653265,1.769592,1069.693878,2469.895102,1.000000,2.247347,980192.414286,580582.510204,358.591837,58.204082


In [4]:
'''
‘region_cd’: ‘지역코드(시도)’,
‘year’: ‘연도’,
‘month’:’월’,
‘building_type’: ‘부동산타입’,
‘tradeprice_sido’ : ‘매매가격지수(시도)’,
‘construction_realized_amount’ : ‘건설기성액(백만원)’,
“cd”: “cd(91일물)“,
‘spirit_deposit_rate’: ‘정기예금금리’,
‘exchange_rate’: ‘환율’,
‘composite_stock_price_index’: ‘종합주가지수’,
‘economy_growth’: ‘경제성장률’,
‘exchequer_bond_three’ : ‘국고채3년‘,
‘household_loan_all’: ‘가계대출액(전국)’,
‘mortgage_all’ : ‘주택대출액(전국)’,
‘numberofnosells’:’미분양 가구수(시도)’,
‘unsalenum_c’:’공사완료후 미분양(민간,시도)’
'''
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

def std_normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        std_value = df[feature_name].std()
        mean_value = df[feature_name].mean()
        result[feature_name] = (df[feature_name] - mean_value) / std_value
    return result


#train = df[(df.year > 2006) & (df.year < 2017) & (df.region_cd == 45000)]
#test = df[df.year >= 2017 & (df.region_cd == 45000)]

train = df[(df.year > 2006) & (df.year < 2017)]
test = df[df.year >= 2017]

#train = normalize(train) # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!
#test = normalize(test)  # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!

train = std_normalize(train) # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!
test = std_normalize(test)  # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!


'''
feature_names = ['region_cd', 'year', 'month', 'building_type',
                 'tradeprice_sido', 'construction_realized_amount','cd',
                 'spirit_deposit_rate','exchange_rate','composite_stock_price_index',
                 'economy_growth','exchequer_bond_three','household_loan_all',
                 'mortgage_all','numberofnosells','unsalenum_c']  # 학습에 사용될 항목들을 설정
'''
feature_names = [
                 'tradeprice_sido','cd',
                 'spirit_deposit_rate','exchange_rate','composite_stock_price_index',
                 'economy_growth','exchequer_bond_three','household_loan_all',
                 'mortgage_all','numberofnosells','unsalenum_c']  # 학습에 사용될 항목들을 설정

input_dim = len(feature_names) # 학습에 사용될 항목들의 갯수 == 한개의 input의 feature 갯수
X_train = train[feature_names]
X_test = test[feature_names]

lable_name = "tradeprice_sido_n1"  # 어떤 값을 예측값으로 넣을것인가 이 경우 다음달 매매가격지수를 예측하도록함
Y_train = train[lable_name]
Y_test = test[lable_name]


print(X_train.shape)
print(X_test.shape)
X_train.head()  #데이터가 어떻게 생겼나 보는 함수

(5560, 11)
(992, 11)


,tradeprice_sido,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,
2010-01-01,-0.054053,0.490385,1.602409,0.764565,-2.397821,3.616001,1.904206,-1.548967,-1.55623,-0.344615,-0.281307
2010-01-01,-1.510399,0.490385,1.602409,0.764565,-2.397821,3.616001,1.904206,-1.548967,-1.55623,1.668217,0.119912
2010-01-01,-0.196832,0.490385,1.602409,0.764565,-2.397821,3.616001,1.904206,-1.548967,-1.55623,1.668217,0.119912
2010-01-01,-1.995848,0.490385,1.602409,0.764565,-2.397821,3.616001,1.904206,-1.548967,-1.55623,1.668217,0.119912
2010-01-01,-1.696012,0.490385,1.602409,0.764565,-2.397821,3.616001,1.904206,-1.548967,-1.55623,1.668217,0.119912


In [5]:
X_test.head()

,tradeprice_sido,cd,spirit_deposit_rate,exchange_rate,composite_stock_price_index,economy_growth,exchequer_bond_three,household_loan_all,mortgage_all,numberofnosells,unsalenum_c
date,,,,,,,,,,,
2017-01-01,3.468766,0.131189,-0.640228,1.549735,-1.81933,0.434124,-1.069946,-1.542133,-1.610761,-0.163232,-0.293776
2017-01-01,1.876317,0.131189,-0.640228,1.549735,-1.81933,0.434124,-1.069946,-1.542133,-1.610761,-0.163232,-0.293776
2017-01-01,-1.397049,0.131189,-0.640228,1.549735,-1.81933,0.434124,-1.069946,-1.542133,-1.610761,0.562747,-0.394042
2017-01-01,2.407133,0.131189,-0.640228,1.549735,-1.81933,0.434124,-1.069946,-1.542133,-1.610761,0.562747,-0.394042
2017-01-01,0.726216,0.131189,-0.640228,1.549735,-1.81933,0.434124,-1.069946,-1.542133,-1.610761,0.562747,-0.394042


## 3. Model & Optimizer

In [6]:
model = nn.Sequential(
            nn.Linear(input_dim,1),
   #         nn.ReLU(),
   #         nn.Linear(200,100),
   #         nn.ReLU(),
   #         nn.Linear(100,50),
   #         nn.ReLU(),
   #         nn.Linear(50,50),
   #         nn.ReLU(),
   #         nn.Linear(50,1),
        )

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [7]:
print(torch.from_numpy(X_train.values))

tensor([[-0.0541,  0.4904,  1.6024,  ..., -1.5562, -0.3446, -0.2813],
        [-1.5104,  0.4904,  1.6024,  ..., -1.5562,  1.6682,  0.1199],
        [-0.1968,  0.4904,  1.6024,  ..., -1.5562,  1.6682,  0.1199],
        ...,
        [ 0.7741, -1.4483, -1.3654,  ...,  2.2106, -0.3446, -0.2813],
        [ 0.5456, -1.4483, -1.3654,  ...,  2.2106, -0.3446, -0.2813],
        [ 0.6456, -1.4483, -1.3654,  ...,  2.2106, -0.3446, -0.2813]],
       dtype=torch.float64)


## 4. Train 

In [8]:
input_data = torch.from_numpy(X_train.values).float()
label = torch.from_numpy(Y_train.values).float()
print(input_data.size())
loss_arr =[]

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(input_data)
    #print("output", output)
    #print("label", label)
    
    loss = loss_func(output,label)
    loss.backward()
    optimizer.step()
    
    loss_arr.append(loss.item())
    print(loss.item())


torch.Size([5560, 11])
0.797967791557312
0.79317706823349
0.7885106801986694
0.7840576767921448
0.7794879674911499
0.7750930190086365
0.7707607746124268
0.7665306329727173
0.7627715468406677
0.7587927579879761
0.7549577951431274
0.7515987753868103
0.7482636570930481
0.7448615431785583
0.7414037585258484
0.7382317781448364
0.7351948618888855
0.7323365211486816
0.7295004725456238
0.7272216081619263
0.7250692844390869
0.7228257656097412
0.7207705974578857
0.7187584638595581
0.7166855931282043
0.7147518992424011
0.7128202319145203
0.711031973361969
0.7091052532196045
0.7073332071304321
0.7056344151496887
0.7042039036750793
0.7027596831321716
0.7012565732002258
0.699824869632721
0.6983675956726074
0.696999728679657
0.695652186870575
0.6943817138671875
0.6931489109992981
0.6919220089912415
0.6906583309173584
0.6894555687904358
0.6882973909378052
0.6872079968452454
0.686066210269928
0.684977114200592
0.6839341521263123
0.6829200983047485
0.681961178779602
0.680971086025238
0.6799648404121399


In [9]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[ 0.0618, -0.0823,  0.1865,  0.0302,  0.0518,  0.0681,  0.0382,  0.0266,
          0.0867, -0.1095,  0.1092]], requires_grad=True), Parameter containing:
tensor([0.2436], requires_grad=True)]


## 5. Visualize Trained Output

In [10]:
'''
‘region_cd’: ‘지역코드(시도)’,
‘year’: ‘연도’,
‘month’:’월’,
‘building_type’: ‘부동산타입’,
‘tradeprice_sido’ : ‘매매가격지수(시도)’,
‘construction_realized_amount’ : ‘건설기성액(백만원)’,
“cd”: “cd(91일물)“,
‘spirit_deposit_rate’: ‘정기예금금리’,
‘exchange_rate’: ‘환율’,
‘composite_stock_price_index’: ‘종합주가지수’,
‘economy_growth’: ‘경제성장률’,
‘exchequer_bond_three’ : ‘국고채3년‘,
‘household_loan_all’: ‘가계대출액(전국)’,
‘mortgage_all’ : ‘주택대출액(전국)’,
‘numberofnosells’:’미분양 가구수(시도)’,
‘unsalenum_c’:’공사완료후 미분양(민간,시도)’
'''

'\n‘region_cd’: ‘지역코드(시도)’,\n‘year’: ‘연도’,\n‘month’:’월’,\n‘building_type’: ‘부동산타입’,\n‘tradeprice_sido’ : ‘매매가격지수(시도)’,\n‘construction_realized_amount’ : ‘건설기성액(백만원)’,\n“cd”: “cd(91일물)“,\n‘spirit_deposit_rate’: ‘정기예금금리’,\n‘exchange_rate’: ‘환율’,\n‘composite_stock_price_index’: ‘종합주가지수’,\n‘economy_growth’: ‘경제성장률’,\n‘exchequer_bond_three’ : ‘국고채3년‘,\n‘household_loan_all’: ‘가계대출액(전국)’,\n‘mortgage_all’ : ‘주택대출액(전국)’,\n‘numberofnosells’:’미분양 가구수(시도)’,\n‘unsalenum_c’:’공사완료후 미분양(민간,시도)’\n'

## 6. Prediction

In [11]:
eval_data = torch.from_numpy(X_test.values).float()
eval_label = torch.from_numpy(Y_test.values).float()

eval_result = model(eval_data)
print(eval_result)
print(torch.nn.functional.sigmoid(eval_result))
print(eval_label)




tensor([[ 0.0743],
        [-0.0242],
        [-0.3170],
        [-0.0818],
        [-0.1857],
        [-0.1912],
        [ 0.2108],
        [ 0.1834],
        [-0.1688],
        [-0.2673],
        [-0.1196],
        [-0.1853],
        [-0.2537],
        [-0.1277],
        [-0.1878],
        [-0.1878],
        [-0.2097],
        [-0.2099],
        [-0.2136],
        [-0.2300],
        [-0.2191],
        [-0.1261],
        [-0.2008],
        [-0.1734],
        [-0.1515],
        [-0.0099],
        [-0.2233],
        [-0.0701],
        [-0.0646],
        [-0.2336],
        [-0.1090],
        [-0.0543],
        [-0.1967],
        [-0.1315],
        [-0.1808],
        [-0.2902],
        [-0.1479],
        [-0.1912],
        [-0.1967],
        [ 0.1889],
        [ 0.1615],
        [-0.2300],
        [-0.1917],
        [-0.2300],
        [-0.2410],
        [-0.1584],
        [-0.2355],
        [-0.0714],
        [-0.0385],
        [ 0.0928],
        [-0.2841],
        [-0.1418],
        [-0.

/Users/lim/anaconda3/envs/NLPApps/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [13]:

def normalize2(df, df_e):
    result = df_e.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df_e[feature_name] - min_value) / (max_value - min_value)
    return result

def std_normalize2(df, df_e):
    result = df_e.copy()
    for feature_name in df_e.columns:
        std_value = df[feature_name].std()
        mean_value = df[feature_name].mean()
        result[feature_name] = (df_e[feature_name] - mean_value) / std_value
    return result


df_e = pd.read_csv("my_data.csv", parse_dates =["date"], index_col ="date")  #엑셀 파일을 읽어 df_e변수에 저장
df_e = df_e.dropna()
test_data = std_normalize2(df, df_e)  # 데이터를 normalize해서 0~1사이 값으로 다 바꿈!

test_input = test_data[feature_names] # 학습에 사용했던 feature들만 뽑음
test_input.head()

test_tensor = torch.from_numpy(test_input.values).float() # pytorch tensor 형태로 데이터 변환

print(model(test_tensor))




tensor([[0.3786],
        [0.3622],
        [0.2453],
        [0.2844],
        [0.2671],
        [0.2662],
        [0.2764],
        [0.2718]], grad_fn=<AddmmBackward>)
